# Santa 2025 - Quick EDA

In [1]:
import pandas as pd
import numpy as np
from decimal import Decimal, getcontext
getcontext().prec = 30

# Load sample submission
df = pd.read_csv('/home/data/sample_submission.csv')
print(f"Shape: {df.shape}")
print(f"Columns: {df.columns.tolist()}")
df.head(10)

Shape: (20100, 4)
Columns: ['id', 'x', 'y', 'deg']


,id,x,y,deg
0,001_0,s0.0,s0.0,s90.0
1,002_0,s0.0,s0.0,s90.0
2,002_1,s0.202736,s-0.511271,s90.0
3,003_0,s0.0,s0.0,s90.0
4,003_1,s0.202736,s-0.511271,s90.0
5,003_2,s0.5206,s0.177413,s180.0
6,004_0,s0.0,s0.0,s90.0
7,004_1,s0.202736,s-0.511271,s90.0
8,004_2,s0.5206,s0.177413,s180.0
9,004_3,s-0.818657,s-0.228694,s180.0


In [ ]:
# Parse the data
df['x'] = df['x'].astype(str).str.lstrip('s').astype(float)
df['y'] = df['y'].astype(str).str.lstrip('s').astype(float)
df['deg'] = df['deg'].astype(str).str.lstrip('s').astype(float)
df['n'] = df['id'].str.split('_').str[0].astype(int)
df['tree_idx'] = df['id'].str.split('_').str[1].astype(int)

print(f"N values: {df['n'].min()} to {df['n'].max()}")
print(f"Total rows: {len(df)}")
print(f"Expected rows: {sum(range(1, 201))} = 1+2+...+200")
print(f"Match: {len(df) == sum(range(1, 201))}")